Import training data provided in Assignment_4_Data_and_Template.xlsx

In [17]:
import pandas as pd
Training_Data = pd.read_excel("Assignment_4_Data_and_Template.xlsx", sheetname="Training Data")
Training_Data.rename(columns={"Unnamed: 7": "Flow2", "Unnamed: 8" :"Flow3"}, inplace=True)
Training_Data.rename(columns={"Unnamed: 10": "Nitrogen2", "Unnamed: 11" :"Nitrogen3"}, inplace=True)
Training_Data.rename(columns={"Unnamed: 13": "Freq2", "Unnamed: 14" :"Freq3"}, inplace=True)

Failure = pd.DataFrame(Training_Data['Failure'])
FType = pd.DataFrame(Training_Data['Type'])

Type_Raw = Training_Data['Type'].copy()
del Training_Data['Failure']
del Training_Data['Type']

Transforming FType to Kessler's Construction
Failure Type 1 = 1,-1,-1,-1,-1 etc...

In [18]:
import numpy as np
Type = pd.DataFrame()

for index,row in FType.iterrows():
    temp = np.ones(6,dtype=int) * -1
    temp[row['Type']-1] = 1
    
    Type[index] = temp

Type = Type.T

Use the training data to build two linear classifiers: (i) a binary classifier, and (ii) a multiclass classifier capable of distinguishing between 6 classes. The sheet titled "Training Data" contains the target class labels (Failure and Type). Use the pseudoinverse method of building linear classifiers. Paste your classifiers in the correct format as shown in the sheet titled "Classifiers"

In [19]:
Training_Data.insert(0, 'Xa', 1)

In [20]:
import numpy as np
Xa = Training_Data.copy()

W_Failure = pd.DataFrame(np.linalg.pinv(Xa).dot(Failure))

W_Type = pd.DataFrame(np.linalg.pinv(Xa).dot(Type))

In [7]:
#Write Results to Excel

import ReadWriteExcel as rw
rw.writeExcelData(W_Failure, "Assignment_4_Data_and_Template.xlsx", "Classifiers", 5,1)
rw.writeExcelData(W_Type, "Assignment_4_Data_and_Template.xlsx", "Classifiers", 5,5)

PermissionError: [Errno 13] Permission denied: 'Assignment_4_Data_and_Template.xlsx'

In [22]:
# Test the classifier on the training data
Train_Out = []

for index,row in Training_Data.iterrows():
    Train_Out.append(np.dot(row,W_Type))
    
Train_Output = []
for row in Train_Out:
    Train_Output.append(list(row).index(max(list(row)))+1)
    
print(Train_Output[0:10])
print(Type_Raw[0:10])

[4, 6, 2, 1, 4, 6, 1, 2, 1, 4]
0    4
1    0
2    2
3    3
4    4
5    0
6    3
7    2
8    5
9    4
Name: Type, dtype: int64


Apply the classifiers to the data provided in the sheet "To be classified." Paste the results into the sheet.

In [7]:
ToClassify = pd.read_excel("Assignment_4_Data_and_Template.xlsx", sheetname="To be classified", skiprows=[0,1,2])

ToClassify.rename(columns={"Unnamed: 7": "Flow2", "Unnamed: 8" :"Flow3"}, inplace=True)
ToClassify.rename(columns={"Unnamed: 10": "Nitrogen2", "Unnamed: 11" :"Nitrogen3"}, inplace=True)
ToClassify.rename(columns={"Unnamed: 13": "Freq2", "Unnamed: 14" :"Freq3"}, inplace=True)

del ToClassify['Failure']
del ToClassify['Type']

In [8]:
Failure_Output = []
Type_Output = []

for index,row in ToClassify.iterrows():
    xa = [1]
    xa.extend(row)

    Failure_Output.append(np.sign(np.dot(xa, W_Failure)).astype(int))
    Type_Output.append(np.dot(xa,W_Type))
    
Type_Out = []

for row in Type_Output:
    Type_Out.append(list(row).index(max(list(row)))+1)

1

In [10]:
#Write results to Excel

import ReadWriteExcel as rw

rw.writeExcelData(pd.DataFrame(Failure_Output), "Assignment_4_Data_and_Template.xlsx", "To be classified", 5, 16)
rw.writeExcelData(pd.DataFrame(Type_Out), "Assignment_4_Data_and_Template.xlsx", "To be classified", 5,17)

Evaluate the performance of your classifiers (using training data alone) using the metrics requested in the sheet "Performance"

For guidance, see slides titled "Step-by-step: Linear Classifier Design" in the lecture on linear classifiers.

In [11]:
#Evaluate Training Data

TD_Fail_Out = []
TD_Type_Output = []

for index,row in Training_Data.iterrows():
    
    TD_Fail_Out.append(np.sign(np.dot(row, W_Failure)).astype(int))
    TD_Type_Output.append(np.dot(row,W_Type))

TD_Type_Out = []
for row in TD_Type_Output:
    TD_Type_Out.append(list(row).index(max(list(row)))+1)

#print (pd.DataFrame(TD_Fail_Out))
#print (pd.DataFrame(TD_Type_Out))

In [12]:
#Make Confusion Matrices

CM_Fail = np.zeros((2,2))
CM_Type = np.zeros((6,6))

for i,r in Failure.iterrows():
    if r[0] == -1:
        if TD_Fail_Out[i] == -1:
            CM_Fail[0][0] += 1
        elif TD_Fail_Out[i] == 1:
            CM_Fail[0][1] += 1
    
    elif r[0] == 1:
        if TD_Fail_Out[i] == 1:
            CM_Fail[1][1] += 1
        elif TD_Fail_Out[i] == -1:
            CM_Fail[1][0] += 1
            
for i, r in FType.iterrows():
    CM_Type[r[0]-1][TD_Type_Out[i]-1] += 1
    
CM_Type = CM_Type.astype(int)

In [13]:
#Calculate accuracy, sensitivity, Specificity, PPV

acc = (CM_Fail[0][0] + CM_Fail[1][1])/(np.sum(CM_Fail))
sen = (CM_Fail[1][1])/(CM_Fail[1][1] + CM_Fail[1][0])
spc = (CM_Fail[0][0])/(CM_Fail[0][0] + CM_Fail[0][1])
ppv = (CM_Fail[1][1]) / (CM_Fail[1][1] + CM_Fail[0][1])

print('Accuracy is ' + str(acc*100) + '%')
print('Sensitivity is ' + str(sen*100) + '%')
print('Specificity is ' + str(spc*100) + '%')
print('PPV is ' + str(ppv*100) + '%')

Accuracy is 81.5%
Sensitivity is 65.1515151515%
Specificity is 97.8484848485%
PPV is 96.803241783%


In [14]:
# Calculate PPV for each of the classes

sums = CM_Type.sum(axis=0)
PPVs = []

for i in range(len(sums)):
    PPVs.append(CM_Type[i][i]/sums[i])

In [15]:
# Write results to Excel

import ReadWriteExcel as rw

rw.writeExcelData(pd.DataFrame(CM_Fail, dtype=int), "Assignment_4_Data_and_Template.xlsx", "Performance", 10,3)
rw.writeExcelData(pd.DataFrame(CM_Type, dtype=int), "Assignment_4_Data_and_Template.xlsx", "Performance", 19,3)
rw.writeExcelData(pd.DataFrame([acc, sen, spc, ppv]), "Assignment_4_Data_and_Template.xlsx", "Performance", 8,7)
rw.writeExcelData(pd.DataFrame([[max(PPVs)*100, PPVs.index(max(PPVs))], [min(PPVs)*100, PPVs.index(min(PPVs))]]), 
                  "Assignment_4_Data_and_Template.xlsx", "Performance", 20,12)